# Contents <a id='back'></a>

* [Introduction](#intro)
    * [Stage 1. Data overview](#data_review)
        * [Data exploration](#data_review_conclusions)
    * [Stage 2. Data preprocessing](#data_preprocessing)
        * [2.1 Column style](#fix_data)
        * [2.2 Missing values](#missing_values)
        * [2.3 Restoring missing values in `total_income`](#na_total_income)
        * [2.4 Restoring missing values in `days_employed`](#na_days_employed)
        * [2.5 Categorization of data](#categorization)
    * [Stage 3. Checking the hypotheses](#hypotheses)
        * [3.1 Hypothesis 1: Is there a correlation between having children and paying back on time?](#hypo1)
        * [3.2 Hypothesis 2: Is there a correlation between family status and paying back on time? ](#hypo2)
        * [3.3 Hypothesis 3: Is there a correlation between income level and paying back on time?](#hypo3)
        * [3.4 Hypothesis 4: How does credit purpose affect the default rate](#hypo4)
    * [General Conclusion](#end)

# Introduction <a id='intro'></a>

Analyzing borrowers’ risk of defaulting:

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

**Hypothesis**

Customer's marital status will impact the customer's decision to default on a loan.
The number of children a customer has will impact the customer's decision to default on a loan.

## Stage 1. Data overview <a id='data_review'></a>

In [1]:
# Loading all the libraries

import pandas as pd

In [2]:
# Load the data

df = pd.read_csv('credit_scoring_eng.csv')
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


### Data exploration <a id='data_review_conclusions'></a>

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

[Now let's explore our data. You'll want to see how many columns and rows it has, look at a few rows to check for potential issues with the data.]

In [3]:
# Let's see how many rows and columns our dataset has

df.shape

(21525, 12)

In [4]:
# let's print the first N rows

display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


[Describe what you see and notice in your printed data sample. Are there any issues that may need further investigation and changes?]

In the table:

1. The value of days_employed appears to be odd because there is no such thing as negative days.
2. The "education" column is inconsistant with using uppercase and lowercase. It needs to be standardize.

In [5]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There is missing values in 'days_employed' and 'total_income' as the total count is less than 21525 (total entries).

In [6]:
# Let's look in the filtered table at the the first column with missing data

df[df['total_income'].isna()].info()


<class 'pandas.core.frame.DataFrame'>
Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


The missing values in 'total_income' column is the same as 'days_employed' column.

In [7]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

df[df['total_income'].isna() | df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Intermediate conclusion**

The number of rows in the filtered table match the number of missing values. Suspect that there is a relation between these 2 columns that led the customers to leave these 2 columns blank. We need to find out the reason.

In [8]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

df_nan = df[df['total_income'].isna()]

In [9]:
# Checking distribution

df_nan['income_type'].value_counts(normalize= True)

income_type
employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: proportion, dtype: float64

**Possible reasons for missing values in data**

1. Related to income type
2. Related to education
3. Related to family status

[Let's start checking whether the missing values are random.]

In [10]:
#Checking distribution in whole dataset

df['income_type'].value_counts(normalize= True)

income_type
employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: proportion, dtype: float64

**Intermediate conclusion**

The overall distribution of both sets of data have similar percentages. Only `retiree` in the df_nan table is 1% higher than the whole data set. Which makes sense because retirees are expected to not be working. However, with similar percentages and no huge differences in the value, the `income_type` might not be the cause of reason for all the missing values.

In [11]:
# Check for other reasons and patterns that could lead to missing values

df_nan['education'].value_counts(normalize= True)

education
secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
Some College           0.003220
SOME COLLEGE           0.003220
Primary Education      0.000460
PRIMARY EDUCATION      0.000460
Name: proportion, dtype: float64

In [12]:
## Checking distribution in whole dataset

df['education'].value_counts(normalize= True)

education
secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
Graduate Degree        0.000046
GRADUATE DEGREE        0.000046
Name: proportion, dtype: float64

**Intermediate conclusion**

From the observation, `secondary education`, `bachelor's degree`, `Some College` and `SOME COLLEGE` have a higher percentage in the df_nan table. However, the percentage is as much as 1% difference. Therefore, the amount is not big enough to put a conclusion that customer's education is a strong reason for the missing values in the columns.

In [13]:
# Checking for other patterns - family_status

df_nan['family_status'].value_counts(normalize= True)

family_status
married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: proportion, dtype: float64

In [14]:
df['family_status'].value_counts(normalize= True)

family_status
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: proportion, dtype: float64

**Conclusions**

Simillar to the previous 2 categories tested for their distribution, 'family_status' of the 2 tables compared are also showing similar percentages. Therefore, the reason for the missing values is not tied to any reasoning and could be the cause of a mistake on the customer's side of not filling in the category.

[Back to Contents](#back)

## Stage 2. Data preprocessing <a id ='data_preprocessing'></a>

Issues to fix:
1. The inconsistant use of uppercase and lowercase in the "education" column.
2. Number of children of each customers
3. The negative value in days_employed
4. And going through other columns to fix irregularities
5. Replace or remove duplicates depending on situations

### Fix data <a id = 'fix_data'></a>

In [15]:
# Check values in education column

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [16]:
# Fix the registers

df['education']= df['education'].str.lower()


In [17]:
# Checking all the values in the column to make sure we fixed them

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

[Check the data the `children` column]

In [18]:
# Let's see the distribution of values in the `children` column

df['children'].value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

It is strange that there is '-1' numbers of children which doesn't makes sense. Besides, '20' children in a household is impressive but considering 76 household has that many children is also strange because we live in a morden era where most households only has 2-3 children on average.

I believe these are cause by human error which they mistyped the value in that column. To solve, lets change the value '20' to '2' and '-1' to '1'.

In [19]:
# [fix the data based on your decision]

df['children'] = df['children'].replace({20:2, -1:1})

In [20]:
# Checking the `children` column again to make sure it's all fixed

df['children'].value_counts()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: count, dtype: int64

In [21]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage

df['days_employed'].value_counts()

days_employed
-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: count, Length: 19351, dtype: int64

The days should be a positive integers. However, the value in the column shows otherwise. This could be a data entry error when typing the values. Better change the negative value into positive.

In [22]:
# Change negative value into positive

df['days_employed'] = df['days_employed'].abs()

In [23]:
# Check the result

df['days_employed'].value_counts()

days_employed
8437.673028      1
3507.818775      1
354500.415854    1
769.717438       1
3963.590317      1
                ..
1099.957609      1
209.984794       1
398099.392433    1
1271.038880      1
1984.507589      1
Name: count, Length: 19351, dtype: int64

In [24]:
# Check the `dob_years` for suspicious values and count the percentage

df['dob_years'].value_counts()

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

There are a total of 101 customers who are age '0', this is abnormal. 

To solve, remove these rows from the dateset, because it is difficult to determine a value to insert that will not affect the overall data. Plus, the data is quite small so it would'nt affect the end result that much.

In [25]:
# Address the issues in the `dob_years` column

df = df[df['dob_years'] != 0]

In [26]:
# Check the result

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]

In [27]:
# Let's see the values for the column

df['family_status'].value_counts()

family_status
married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: count, dtype: int64

There is no problematic values in 'family_status'.

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

In [28]:
# Let's see the values in the column

df['gender'].value_counts()

gender
F      14164
M       7259
XNA        1
Name: count, dtype: int64

1 customer did not fill in their gender. Let's remove that row from the dataset as well.

In [29]:
# Remove 'XNA'

df = df[df['gender'] != 'XNA']

In [30]:
# Check the result

df['gender'].value_counts()

gender
F    14164
M     7259
Name: count, dtype: int64

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]

In [31]:
# Let's see the values in the column

df['income_type'].value_counts()

income_type
employee                       11064
business                        5064
retiree                         3836
civil servant                   1453
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

The values like 'entrepreneur' should be rename to **'business'** since they are similar and should be categorise as one. 'paternity / maternity leave' should be rename to **'employee'** since they are technically employees but on leave. 

Use groupby with 'dob_years' to check if unemployed = retiree and if student is underage so it could be in 'unemployed'.

In [32]:
# Renaming the income type

df['income_type'] = df['income_type'].replace({'entrepreneur':'business' , 'paternity / maternity leave': 'employee'})

In [33]:
df.groupby('income_type')['dob_years'].value_counts()

income_type  dob_years
business     35           192
             38           180
             40           178
             31           177
             41           175
                         ... 
retiree      24             1
             22             1
student      22             1
unemployed   31             1
             45             1
Name: count, Length: 214, dtype: int64

It seems that the unemployed customer's age is less than 50 years old. Therefore, they could not be categorise as 'retiree'.

The customer who is a student is 22 years old. Therefore, they are legally off age to work and receive income, just that they wish to continue studies. So, these 2 values will remain the same.

In [34]:
# Check the result

df['income_type'].value_counts()

income_type
employee         11065
business          5066
retiree           3836
civil servant     1453
unemployed           2
student              1
Name: count, dtype: int64

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

In [35]:
# Checking duplicates

print(df.duplicated().sum())

71


In [36]:
# Address the duplicates, if they exist

df = df.drop_duplicates().reset_index(drop = True)

In [37]:
# Last check whether we have any duplicates

print(df.duplicated().sum())

0


In [38]:
# Check the size of the dataset that you now have after your first manipulations with it

df.shape

(21352, 12)

**Intermediate conclusion**

There are a total of 71 duplicates in this data. I've decided to drop all duplicates as the amount is less than 1% and will not affect the overall data. Current count of the total number of customers left is 21352.

[Back to Contents](#back)

### Missing values <a id='missing_values'></a>

In [39]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Based on the list of columns above, education and education_id is related. To remove duplicates, use dictionary to combine data of these 2 columns and catagorise them accordingly.

In [40]:
# Find the dictionaries

edu_dictionary = df[['education', 'education_id']]
display (edu_dictionary)

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,secondary education,1
3,secondary education,1
4,secondary education,1
...,...,...
21347,secondary education,1
21348,secondary education,1
21349,secondary education,1
21350,secondary education,1


In [41]:
edu_dictionary = edu_dictionary.drop_duplicates().reset_index(drop = True)
display (edu_dictionary)

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


**Intermediate conclusion**

Now we can clearly see that the education in this dataset is categorize in 5 types and represented by education_id from 0 - 4.

### Restoring missing values in `total_income` <a id='na_total_income'></a>

1. To start, we have to create a function to categorize the age column and insert it as a new column in the dataset.
2. Then, using this age category, paired with income type to identify the Mean and Median for each category.
3. After that, decide if either Mean or Median is more suitable to replace the missing values in total_income.

In [42]:
# Let's write a function that calculates the age category

def age_category(age):
    if age < 25:
        return '<25'
    if age < 35:
        return '25 - 34'
    if age < 45:
        return '35 - 44'
    else:
        return '> 44'

In [43]:
# Test if the function works
age_category (35)

'35 - 44'

In [44]:
# Creating new column based on function

df ['age_group'] = df['dob_years'].apply(age_category)
display (df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,35 - 44
21348,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 44
21349,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,35 - 44
21350,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,35 - 44


In [45]:
# Checking how values in the new column

df ['age_group'].value_counts()

age_group
> 44       9646
35 - 44    5740
25 - 34    5092
<25         874
Name: count, dtype: int64

[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]

[Create a table that only has data without missing values. This data will be used to restore the missing values.]

In [46]:
# Create a table without missing values and print a few of its rows to make sure it looks fine

data_x_nan = df[df['total_income'].isnull()!= True]
data_x_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44


In [47]:
# Look at the mean values for income based on your identified factors

data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'total_income', aggfunc='mean')

age_group,25 - 34,35 - 44,<25,> 44
income_type,,,,
business,31270.520395,33992.827968,25446.057616,32960.933471
civil servant,27026.159686,27836.571764,23548.822930,27550.495883
employee,25621.575071,26492.515800,21288.654010,26020.069560
retiree,19271.650143,23344.197000,14298.976000,21926.734880
student,NaN,NaN,15712.260000,NaN
unemployed,9593.119000,NaN,NaN,32435.602000


In [48]:
# Look at the median values for income based on your identified factors

data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'total_income', aggfunc='median')

age_group,25 - 34,35 - 44,<25,> 44
income_type,,,,
business,27221.1265,28489.5290,22189.078,28379.2085
civil servant,24436.2400,24008.6225,21361.730,23836.6080
employee,22722.2705,23226.3360,19670.104,22873.1320
retiree,15378.5635,19910.4685,14298.976,18962.3180
student,NaN,NaN,15712.260,NaN
unemployed,9593.1190,NaN,NaN,32435.6020


**Explaination**

There is a significant outliner between the Mean and the Median values for total income. In this case, I have decided to use Median for a more accurate protrayal in the dataset. Each missing value will be replace by the Median values base on their age group and income type.


In [49]:
#  Write a function that we will use for filling in missing values
        
pivot_for_total_income = data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'total_income', aggfunc='median')

def get_median_total_income (x):
    age_group = x ['age_group']
    income_type = x ['income_type']
    try:
        return pivot_for_total_income[age_group][income_type]
    except:
        return 'error'

In [50]:
# Check if it works

pivot_for_total_income['35 - 44']['employee']

23226.336

In [51]:
# Apply it to every row

df['median_total_income'] = df.apply(get_median_total_income, axis = 1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44,23226.3360
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44,23226.3360
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34,22722.2705
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34,22722.2705
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44,18962.3180


In [52]:
# Check if we got any errors

df[df['median_total_income']== 'error'] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income


[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]


In [53]:
# Replacing missing values if there are any errors

df['total_income']= df['total_income'].fillna(df['median_total_income'])

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [54]:
# Checking the number of entries in the columns

df.loc[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income


###  Restoring values in `days_employed` <a id='na_days_employed'></a>

1. We will use back the age column created previously and pair with days_employed to identify the Mean and Median for each income category.
2. Then we decide if either Mean or Median is more suitable to replace the missing values in days_employed`.

In [55]:
# Distribution of `days_employed` medians based on your identified parameters

data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'days_employed', aggfunc='median')


age_group,25 - 34,35 - 44,<25,> 44
income_type,,,,
business,1168.374165,1693.627478,691.676752,2069.179391
civil servant,1864.901726,3199.078647,922.372533,3642.738847
employee,1274.074208,1748.001959,769.758975,2175.082953
retiree,366452.103867,360062.669279,334764.259831,365251.578178
student,NaN,NaN,578.751554,NaN
unemployed,337524.466835,NaN,NaN,395302.838654


In [56]:
# Distribution of `days_employed` means based on your identified parameters

data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'days_employed', aggfunc='mean')

age_group,25 - 34,35 - 44,<25,> 44
income_type,,,,
business,1480.207464,2141.167652,787.090016,2863.040368
civil servant,2003.871330,3570.577219,975.347977,4521.299143
employee,1582.711337,2389.456051,873.864193,3127.783023
retiree,364117.596682,364174.061366,334764.259831,365044.096799
student,NaN,NaN,578.751554,NaN
unemployed,337524.466835,NaN,NaN,395302.838654


**Explaination**

Median is more suitable than Mean as there is significant outliers in their amounts.

In [57]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter

pivot_for_days_employed = data_x_nan.pivot_table(index = 'income_type', columns = 'age_group', values = 'days_employed', aggfunc='median')

def get_median_days_employed (x):
    age_group = x ['age_group']
    income_type = x ['income_type']
    try:
        return pivot_for_days_employed[age_group][income_type]
    except:
        return 'error'

In [58]:
# Check that the function works

pivot_for_days_employed['<25']['civil servant']

922.3725334756526

In [59]:
# Apply function to the income_type

df['median_days_employed'] = df.apply(get_median_days_employed, axis = 1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income,median_days_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44,23226.3360,1748.001959
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44,23226.3360,1748.001959
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34,22722.2705,1274.074208
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34,22722.2705,1274.074208
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44,18962.3180,365251.578178


In [60]:
# Check if function worked

df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income,median_days_employed
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,18962.3180,to have a wedding,> 44,18962.3180,365251.578178
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,24008.6225,education,35 - 44,24008.6225,3199.078647
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,18962.3180,building a real estate,> 44,18962.3180,365251.578178
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,23836.6080,second-hand car purchase,> 44,23836.6080,3642.738847
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,18962.3180,to have a wedding,> 44,18962.3180,365251.578178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21316,2,NaN,47,secondary education,1,married,0,M,business,0,28379.2085,purchase of a car,> 44,28379.2085,2069.179391
21322,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,22873.1320,wedding ceremony,> 44,22873.1320,2175.082953
21324,0,NaN,48,bachelor's degree,0,married,0,F,business,0,28379.2085,building a property,> 44,28379.2085,2069.179391
21329,1,NaN,42,secondary education,1,married,0,F,employee,0,23226.3360,building a real estate,35 - 44,23226.3360,1748.001959


In [61]:
# Replacing missing values

df['days_employed']= df['days_employed'].fillna(df['median_days_employed'])

In [62]:
# Check the entries in all columns - make sure we fixed all missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21352 non-null  int64  
 1   days_employed         21352 non-null  float64
 2   dob_years             21352 non-null  int64  
 3   education             21352 non-null  object 
 4   education_id          21352 non-null  int64  
 5   family_status         21352 non-null  object 
 6   family_status_id      21352 non-null  int64  
 7   gender                21352 non-null  object 
 8   income_type           21352 non-null  object 
 9   debt                  21352 non-null  int64  
 10  total_income          21352 non-null  float64
 11  purpose               21352 non-null  object 
 12  age_group             21352 non-null  object 
 13  median_total_income   21352 non-null  float64
 14  median_days_employed  21352 non-null  float64
dtypes: float64(4), int6

[Back to Contents](#back)

### Categorization of data <a id='categorization'></a>

Text data:
1. Categorize purpose column as there are many purpose that are similar in nature. Categorization makes it easier to see the general purpose of the customers and to see if it is related to their credit score later on.

Numerical data:
1. Categorize total_income into groups such as lower, middle or higher income group makes it easier to see which income groups has a higher credit score later on.

In [63]:
# Print the values for your selected data for categorization

df['purpose']

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21347       housing transactions
21348          purchase of a car
21349                   property
21350          buying my own car
21351               to buy a car
Name: purpose, Length: 21352, dtype: object

[Let's check unique values]

In [64]:
# Check the unique values

df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

[What main groups identify based on the unique values?]

From this list, we can deduce the 4 main purposes which is 'real estate', 'car', 'education' and 'wedding'


In [65]:
# Let's write a function to categorize the data based on common topics

category = {
    'purchase of the house': 'real estate',
    'car purchase': 'car',
    'supplementary education': 'education',
    'to have a wedding': 'wedding',
    'housing transactions': 'real estate',
    'education': 'education',
    'having a wedding': 'wedding',
    'purchase of the house for my family': 'real estate',
    'buy real estate': 'real estate', 
    'buy commercial real estate': 'real estate',
    'buy residential real estate': 'real estate',
    'construction of own property': 'real estate',
    'property': 'real estate',
    'building a property': 'real estate',
    'buying a second-hand car': 'car',
    'buying my own car': 'car',
    'transactions with commercial real estate': 'real estate',
    'building a real estate': 'real estate',
    'housing': 'real estate',
    'transactions with my real estate': 'real estate',
    'cars': 'car', 
    'to become educated': 'education',
    'second-hand car purchase': 'car',
    'getting an education': 'education',
    'car': 'car',
    'wedding ceremony': 'wedding',
    'to get a supplementary education': 'education',
    'purchase of my own house': 'real estate',
    'real estate transactions': 'real estate',
    'getting higher education': 'education',
    'to own a car': 'car',
    'purchase of a car': 'car',
    'profile education': 'education',
    'university education': 'education',
    'buying property for renting out': 'real estate',
    'to buy a car': 'car',
    'housing renovation': 'real estate',
    'going to university': 'education'
}

df['purpose_category'] = df['purpose'].replace(category)

In [66]:
# Create a column with the categories and count the values for them

df['purpose_category'].value_counts()

purpose_category
real estate    10763
car             4284
education       3995
wedding         2310
Name: count, dtype: int64

[If you decide to categorize the numerical data, you'll need to come up with the categories for it too.]

In [67]:
# Looking through all the numerical data in your selected column for categorization

df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'age_group', 'median_total_income',
       'median_days_employed', 'purpose_category'],
      dtype='object')

**Explaination**

Let's categorize `total_income`.

In [68]:
# Getting summary statistics for the column

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income,median_days_employed,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44,23226.3360,1748.001959,real estate
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44,23226.3360,1748.001959,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34,22722.2705,1274.074208,real estate
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34,22722.2705,1274.074208,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44,18962.3180,365251.578178,wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,25 - 34,27221.1265,1168.374165,real estate
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,35 - 44,28489.5290,1693.627478,real estate
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,> 44,22873.1320,2175.082953,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,35 - 44,23226.3360,1748.001959,wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,35 - 44,23226.3360,1748.001959,real estate


**Explaination**

Based on observation, there are ranges of 10K, 20K, 30K and 40K. Therefore, categorize around 20K difference for each category:

1. Below 20K - 'low income'
2. Below 40K - 'middle income'
3. 40K and above - 'high income'

In [69]:
# Creating function for categorizing into different numerical groups based on ranges

def income_category(income):
    
    if income < 20000:
        return 'low income'
    if income < 40000:
        return 'middle income'
    else:
        return 'high income'

In [70]:
# Creating column with categories

df['income_group'] = df['total_income'].apply(income_category)
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income,median_days_employed,purpose_category,income_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,35 - 44,23226.3360,1748.001959,real estate,high income
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,35 - 44,23226.3360,1748.001959,car,low income
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,25 - 34,22722.2705,1274.074208,real estate,middle income
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,25 - 34,22722.2705,1274.074208,education,high income
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,> 44,18962.3180,365251.578178,wedding,middle income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,35 - 44,28489.5290,1693.627478,real estate,middle income
21348,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 44,18962.3180,365251.578178,car,middle income
21349,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,35 - 44,23226.3360,1748.001959,real estate,low income
21350,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,35 - 44,23226.3360,1748.001959,car,middle income


In [71]:
# Count each categories values to see the distribution

df['income_group'].value_counts()

income_group
middle income    10774
low income        7779
high income       2799
Name: count, dtype: int64

[Back to Contents](#back)

## Checking the Hypotheses <a id='hypotheses'></a>


### Hypothesis 1: Is there a correlation between having children and paying back on time? <a id='hypo1'></a>

In [72]:
# Check the children data and paying back on time

children_pivot = df.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')
print(children_pivot)

debt            0       1
children                 
0         12963.0  1058.0
1          4397.0   442.0
2          1912.0   202.0
3           301.0    27.0
4            37.0     4.0
5             9.0     NaN


In [73]:
# Calculating default-rate based on the number of children

children_pivot['default_rate']= children_pivot[1] / (children_pivot[1] + children_pivot[0])*100
children_pivot.sort_values(by= 'default_rate', ascending=True)

debt,0,1,default_rate
children,,,
0,12963.0,1058.0,7.545824
3,301.0,27.0,8.231707
1,4397.0,442.0,9.134119
2,1912.0,202.0,9.555345
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Conclusion**

The default rate for each category is the the total number of defaulters (1) in that column in a category divided by the total number of instances column (1 & 0) in that category.

Example, to calculate the default rate for instances where `children` is equal to `0`;

$$ ~default \;rate = {\mathrm{total\; count \; of \; \textbf{pay_late} \; when \; \textbf{children} \; is  \;equal \;to \;0} \over \mathrm{total\; count \; of \; \textbf{pay_late} + \; \textbf{pay_on_time} \; when \; \textbf{children} \; is  \;equal \;to \; 0} }~$$


We can see that customers with ' 1 ',' 2 ' and ' 4 ' children has a higher default rate at around 9%. However, to prove that with having more children, customers will have a higher chance of defaulting their loan, customers with ' 3 ' and ' 5 ' children should have a higher default rate as well.

But, this table proved otherwise. Therefore, I believe the number of children does not affect the customers to default their loan.

In [74]:
testing = df.pivot_table(index='children', values='debt', aggfunc='mean')
print(testing)

              debt
children          
0         0.075458
1         0.091341
2         0.095553
3         0.082317
4         0.097561
5         0.000000


### Hypothesis 2: Is there a correlation between family status and paying back on time? <a id='hypo2'></a>

In [75]:
# Check the family status data and paying back on time

family_pivot = df.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')
print(family_pivot)

debt                   0    1
family_status                
civil partnership   3743  386
divorced            1100   85
married            11363  927
unmarried           2521  273
widow / widower      892   62


In [76]:
# Calculating default-rate based on family status

family_pivot['default_rate']= family_pivot[1] / (family_pivot[1] + family_pivot[0])*100
family_pivot.sort_values(by= 'default_rate', ascending=True)

debt,0,1,default_rate
family_status,,,
widow / widower,892,62,6.498952
divorced,1100,85,7.172996
married,11363,927,7.542718
civil partnership,3743,386,9.348511
unmarried,2521,273,9.770938


**Conclusion**

To calculate the default rate for each family status;

$$ ~default \;rate = {\mathrm{total\; count \; of \; \textbf{pay_late} \;[0] \; column} \over \mathrm{total\; count \; of \; \textbf{pay_late} \; [1] \; column + \; \textbf{pay_on_time} \; [0] \; column} }~$$


We can see that customers who are 'unmarried' in their status has a higher chance to default their loan. Customers who are in a 'civil partnership' also ranks high in their default rate and both status are around 9%.

Therefore, there is a possible relationship that 'unmarried' customers and customers who are in a 'civil partnership' have a higher chance of defaulting their loans.

### Hypothesis 3: Is there a correlation between income level and paying back on time? <a id='hypo3'></a>

In [77]:
# Check the income level data and paying back on time

income_pivot = df.pivot_table(index='income_group', columns='debt', values='days_employed', aggfunc='count')
print(income_pivot)

# Calculating default-rate based on income level

income_pivot['percent_1']= income_pivot[1] / (income_pivot[1] + income_pivot[0])*100
income_pivot.sort_values(by= 'percent_1', ascending=True)

debt              0    1
income_group            
high income    2606  193
low income     7134  645
middle income  9879  895


debt,0,1,percent_1
income_group,,,
high income,2606,193,6.895320
low income,7134,645,8.291554
middle income,9879,895,8.307035


**Conclusion**


The customers from middle income and low income group has the highest percentage of around 8% in not paying their debt on time. Which makes sense because there are often financial struggles in these age group. 

Therefore, there is a correlation between income level and paying back on time.


### Hypothesis 4: How does credit purpose affect the default rate? <a id='hypo4'></a>

In [78]:
# Check the percentages for default rate for each credit purpose and analyze them

purpose_pivot = df.pivot_table(index='purpose_category', columns='debt', values='days_employed', aggfunc='count')
print(purpose_pivot)

purpose_pivot['percent_1']= purpose_pivot[1] / (purpose_pivot[1] + purpose_pivot[0])*100
purpose_pivot.sort_values(by= 'percent_1', ascending=True)


debt                 0    1
purpose_category           
car               3884  400
education         3625  370
real estate       9984  779
wedding           2126  184


debt,0,1,percent_1
purpose_category,,,
real estate,9984,779,7.237759
wedding,2126,184,7.965368
education,3625,370,9.261577
car,3884,400,9.337068


**Conclusion**

It seems that customers who use bank loans on education and cars has a high percentage of around 9% of not paying back on time.
Both category seems to have almost the same amout of total customers but they rank the highest in the percentage.

This data shows a strong correlation between credit purpose and paying back on time.

[Back to Content](#back)

## General Conclusion <a id='end'></a>

The initial 2 hypothesis made by this bank is :
1. Customer’s marital status has an impact on whether they will default on a loan.
2. Customer’s number of children has an impact on whether they will default on a loan. 

However, when we pair the number of children with their debt percentage, we can see that the number of children does not appear in ascending order when compare to the debt percentage. Plus, even the highest number of children is '5' but there are no customer who pays back their debt late. 

Therefore, the second hypothesis is False and there is no correlation between having children and paying back on time.

As for customer's maritial status, it seems that unmarried customers which is those 'unmarried' or in a 'civil relationship' has a higher percentage of not paying their debt on time. Both are around 9%. It probably shows people who have not gone through marriage will be more irresponsible in their finances and debt management.

Therefore, there might be a correlation between family status and paying back on time.

What I would like to point out:

I feel the purpose of the customer's borrowing a loan might be more accurate to evaluate the ability of a potential borrower to repay their loan.

The reason is because customer's borrowing a loan just makes more sense in this dataset in terms of how their purpose will affect the customers to payback their loans. 

1. Purchasing real estates give a ROI (return in investment),thus encouraging customers to payback their loan. 
2. Weddings are sometimes subsidised with congratulatory money from the guest at the wedding also lessen the cost on weddings and more customers tend to payback on time.
3. Customers who borrows for education might not have enough energy making enough income to payback their loans. And education is an investment for the future which means the result to higher income will only happen in the future.
4. Customers who borrows for purchasing cars are just purchasing liabilities that the value of car will only decrease as time goes by. Therefore, it does not generate income fast enough for the customers to repay back their loan.

Also, the comparison of customer's purpose to debt percentage have a similar total number of customers (around 4000) and they are rank at the highest percentage of paying their debt later. This shows a similar set of numbers that will not drastically affect the condition during the comparison of the data.

In conclusion, the initial 2 hypothesis made in the begining, only 1 is proven.


[Back to Content](#back)